<a href="https://colab.research.google.com/github/Ya-Andy/real_DS/blob/master/pr6_autoru_webscrap_V99.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import csv
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
import re
from datetime import datetime
from tqdm import tqdm

In [18]:
from IPython.display import display
pd.options.display.max_columns = None

In [19]:
BRANDS = ['bmw', 'volkswagen', 'nissan', 'mercedes', 'toyota', 'audi', 'mitsubishi', 'skoda', 'volvo', 'honda', 'infiniti', 'lexus']

HEADERS = {'accept': '*/*',
    #'Cache-Control': 'max-age=0',
    #'Connection': 'keep-alive',
    'Cookie': 'autoru_gdpr=1;'
    #'Cookie': 'autoru_gdpr=1; _csrf_token=5dfa61e4bddf625cd6dc29bb4f26de55ab3f85688d2ab79b; autoru_sid=a%3Ag5fccb8b12p59a82c54a8cc4i3np1f9c.09cd4e88cad240cd5e3f862864682fe2%7C1607252145026.604800.IAcKVXxiiUVZ-X6UP8xd_Q.GsWliKqzHmzDrD8GnsHnLa9BeIJekmrqlZdDLgO3Pxg; autoruuid=g5fccb8b12p59a82c54a8cc4i3np1f9c.09cd4e88cad240cd5e3f862864682fe2; suid=3bff210bd0089cb4a659e9cdda4d7870.6dc0ef408f841efe19590747a9e53d59; yuidcs=1; yuidlt=1; yandexuid=47164981607251580; crookie=6A0pGL2ES93F26JiwAjo887kgY+fhuycefFOTU1ykGF1V6ew7rdsn4GLhekjSLBEJgBLN/Bh5gbnYvJTBWqE1jVQEpU=; cmtchd=MTYwNzI1MjEzOTYxNg==; proven_owner_popup=closed; gids=213; gradius=200; cycada=o4BSnmOKYym8FfpnTkBJDrIestxdzbAOSCWEC9Pg9Tk=; _ym_uid=1607252140335333202; _ym_d=1607592387; _ym_isad=2; gdpr=0; from=direct; from_lifetime=1607592387762; X-Vertis-DC=sas; _ym_visorc_22753222=b; _ym_visorc_148422=w; _ym_visorc_148383=w',
    #'Connection': 'keep-alive',
    #'Host': 'auto.ru',
    #'Upgrade-Insecure-Requests': '1',
    #'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:78.0) Gecko/20100101 Firefox/78.0', 
}


In [20]:
def search_url(brand):
    '''
    создаёт адрес для поиска предложений по марке автомобиля
    имеет возможность выбора места поиска (в н.в. Москва) и
    состояния автомобиля: новый, подержанный или все предложения
    
    '''
    
    location = 'sankt-peterburg'    # 'moskva' or anything else
    condition = 'used'   # 'new', 'all'
    
    url = f"https://auto.ru/{location}/cars/{brand}/{condition}/"
    
    return url



In [21]:
def get_html(url, params=None):
    '''
    выполнение запроса get по сформированному адресу
    с возможностью передать доп. параметры в аргументах
    
    '''
    
    r = requests.get(url, headers=HEADERS, params=params)
    
    return r



In [22]:
def pages_count(soup):
    ''' 
    определяет общее количество страниц в поисковой выдаче
    для последующей организации цикла перебора всех страниц
    
    '''
    
    pagination = soup.find_all('a', class_='ListingPagination-module__page')
    
    print(f'pages: {len(pagination)}')
    return int(pagination[-1].get_text())
    
    

In [23]:
def all_reff(brand):
    '''
    создаёт список из всех ссылок на автомобили в продаже
    
    '''
    
    URL = search_url(brand)
    #print(URL)
    response = get_html(URL)
    if response.status_code != 200:
        raise BaseException("response code " + str(response.status_code))
    soup = BeautifulSoup(response.content, 'html.parser')
    #print(soup)
    pages = pages_count(soup)
    
    offer_url = []
    
    for i in range(pages):
        params={'page': i+1} 
        print('Collecting offers for {} page {} of {}...'.format(brand.upper(), i+1, pages))
        response = get_html(URL, params=params)
        soup = BeautifulSoup(response.content, 'html.parser')
        boxes = soup.find_all('div', class_="ListingItem-module__container")
        for box in boxes:
            car_url = box.find('a', class_='Link ListingItemTitle-module__link').attrs['href']
            offer_url.append(car_url)
        time.sleep(0.3)
        
    return offer_url

#all_reff('MITSUBISHI')

In [24]:
def parsing_page_one_ad(url):
    '''
    Функция по извлечению данных со страницы объявления в словарь им. А.Беломойкина
    
    '''
    
    print('get.. ', end = '')
    response = get_html(url)
    response.encoding ='utf8'
    print(f'{response.status_code} ', end = '')
    if response.status_code != 200:
        raise BaseException("response code " + str(response.status_code))
    page = BeautifulSoup(response.text, 'html.parser')
    data_dict = {}
    data_dict['car_url'] = url
    data_dict['parsing_unixtime'] = int(time.time())
    
# в разделах script ищем вхождение элемента и присваиваем А содержимое скрипта, как строку
# далее в А находим сам элемент, н-р 'complectation":', и содержимое между {}. 
# Отсекаем начало, оставляя только содержимое словаря
    
    print('1.. ', end = '')
    for script in page.find_all("script"):
        if 'complectation":{"id"' in str(script):
            a = str(script)   
            data_dict['complectation_dict'] = re.search(r'complectation":{"id.*?}', a)[0][15:]
        if 'equipment":{' in str(script):
            a = str(script)  
            data_dict['equipment_dict'] = re.search(r'equipment":{.*?}', a)[0][11:]
        if '{"mileage":' in str(script):
            a = str(script)  
            data_dict['mileage'] = re.search(r'"mileage":\d*', a)[0][9:]
        if '"model_info":' in str(script):
            a = str(script)  
            data_dict['model_info'] = re.search(r'"model_info":{.*?}', a)[0][13:]
            data_dict['model_name'] = re.search(r'model_info":{"code":".*?"', a)[0][20:].strip('"')
        if 'super_gen":{' in str(script):
            a = str(script)  
            data_dict['super_gen'] = re.search(r'super_gen":{.*?}', a)[0][11:]
        if 'vendor":"' in str(script):
            a = str(script)  
            data_dict['vendor'] = re.search(r'vendor":".*?"', a)[0][9:].strip('"')
    
    print('2.. ', end = '')
    for tag in page.find_all('div'):
        if tag.get("title") == "Идентификатор объявления":
            data_dict['sell_id'] = re.search(r'\d+', tag.text)[0]
    
    print('3.. ', end = '')
    for tag in page.find_all("meta"):
        if tag.get("itemprop") == "bodyType":
            data_dict['bodyType'] = tag.get("content")
        if tag.get("itemprop") == "brand":
            data_dict['brand'] = tag.get("content")
        if tag.get("itemprop") == "color":
            data_dict['color'] = tag.get("content")
        if tag.get("itemprop") == "description":
            data_dict['description'] = tag.get("content")
        if tag.get("itemprop") == "engineDisplacement":
            data_dict['engineDisplacement'] = tag.get("content")
        if tag.get("itemprop") == "enginePower":
            data_dict['enginePower'] = tag.get("content")
        if tag.get("itemprop") == "fuelType":
            data_dict['fuelType'] = tag.get("content")
        if tag.get("itemprop") == "image":
            data_dict['image'] = tag.get("content")
        if tag.get("itemprop") == "modelDate":
            data_dict['modelDate'] = tag.get("content")
        if tag.get("itemprop") == "name":
            data_dict['name'] = tag.get("content")
        if tag.get("itemprop") == "numberOfDoors":
            data_dict['numberOfDoors'] = tag.get("content")
        if tag.get("itemprop") == "price":
            data_dict['price'] = tag.get("content")
        if tag.get("itemprop") == "priceCurrency":
            data_dict['priceCurrency'] = tag.get("content")
        if tag.get("itemprop") == "productionDate":
            data_dict['productionDate'] = tag.get("content")
        if tag.get("itemprop") == "vehicleConfiguration":
            data_dict['vehicleConfiguration'] = tag.get("content")
        if tag.get("itemprop") == "vehicleTransmission":
            data_dict['vehicleTransmission'] = tag.get("content")
    
    print('4.. ', end = '')
    CardInfoRow = page.find_all('span', class_="CardInfoRow__cell")
    for i, tag in enumerate (CardInfoRow):
        if tag.text == "Владельцы":
            data_dict['Владельцы'] = CardInfoRow[i+1].text.replace(u'\xa0', u' ') # заменяем юникодовский пробел
        if tag.text == "Владение":
            data_dict['Владение'] = CardInfoRow[i+1].text.replace(u'\xa0', u' ') 
        if tag.text == "ПТС":
            data_dict['ПТС'] = CardInfoRow[i+1].text.replace(u'\xa0', u' ') 
        if tag.text == "Привод":
            data_dict['Привод'] = CardInfoRow[i+1].text.replace(u'\xa0', u' ') 
        if tag.text == "Руль":
            data_dict['Руль'] = CardInfoRow[i+1].text.replace(u'\xa0', u' ') 
        if tag.text == "Состояние":
            data_dict['Состояние'] = CardInfoRow[i+1].text.replace(u'\xa0', u' ') 
        if tag.text == "Таможня":
            data_dict['Таможня'] = CardInfoRow[i+1].text.replace(u'\xa0', u' ') 
    
    return data_dict



In [25]:
# собственно попытка сбора информации

for brand in BRANDS:
    all_offers = all_reff(brand)
    
    print('All offers collected', len(all_offers))
    print('Start parsing for', brand.upper())
    
    df = pd.DataFrame()

    for idx, ad_url in enumerate(all_offers):
        print(f'{idx} ', end = '')
        try:
            df = df.append(parsing_page_one_ad(ad_url), ignore_index=True)
            print('sleep.. ', end = '')
            time.sleep(0.2)
            print('finish')
        except:
            pass
    
    print('The resulting dataset has size: ', df.shape)
    
    FILENAME = brand.upper() + datetime.strftime(datetime.now(), "%Y%m%d") + "_train.csv"      
    df.to_csv(FILENAME, index = False)
    print('Dataset written to file {}'.format(FILENAME))
    time.sleep(0.3)
    

Выходные данные были обрезаны до нескольких последних строк (5000).
834 get.. 200 1.. 2.. 3.. 4.. sleep.. finish
835 get.. 200 1.. 2.. 3.. 4.. sleep.. finish
836 get.. 200 1.. 2.. 3.. 4.. sleep.. finish
837 get.. 200 1.. 2.. 3.. 4.. sleep.. finish
838 get.. 200 1.. 2.. 3.. 4.. sleep.. finish
839 get.. 200 1.. 2.. 3.. 4.. sleep.. finish
840 get.. 200 1.. 2.. 3.. 4.. sleep.. finish
841 get.. 200 1.. 2.. 3.. 4.. sleep.. finish
842 get.. 200 1.. 2.. 3.. 4.. sleep.. finish
843 get.. 200 1.. 2.. 3.. 4.. sleep.. finish
844 get.. 200 1.. 2.. 3.. 4.. sleep.. finish
845 get.. 200 1.. 2.. 3.. 4.. sleep.. finish
846 get.. 200 1.. 2.. 3.. 4.. sleep.. finish
847 get.. 200 1.. 2.. 3.. 4.. sleep.. finish
848 get.. 200 1.. 2.. 3.. 4.. sleep.. finish
849 get.. 200 1.. 2.. 3.. 4.. sleep.. finish
850 get.. 200 1.. 2.. 3.. 4.. sleep.. finish
851 get.. 200 1.. 2.. 3.. 4.. sleep.. finish
852 get.. 200 1.. 2.. 3.. 4.. sleep.. finish
853 get.. 200 1.. 2.. 3.. 4.. sleep.. finish
854 get.. 200 1.. 2.. 3.. 4.. sl

In [2]:
# подключение гугл-диска
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [15]:
df.to_csv("T_train.csv", index=False)

In [16]:
# передача файла из колаба на локальную машину
from google.colab import files

with open('example.txt', 'w') as f:
  f.write('some content')

files.download('example.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# from joblib import Parallel, delayed
# def collect(ad_url):
#     df.append(parsing_page_one_ad(ad_url), ignore_index=True)
# df = Parallel(n_jobs = 2)(delayed(collect)(ad_url) for ad_url in tqdm(list(all_offers)))